## 상태 전이표로부터 코드 생성 접근하기

In [ ]:
# 초기 상태를 설정
state = "초기 상태"
total_amount = 0
price = 1200
stock = 5

# 상태 전이 함수 정의
def transition(event):
    global state, total_amount, price, stock

    if state == "초기 상태":
        if event == "동전 투입":
            coin = int(input("동전을 투입해주세요: "))
            total_amount += coin
            state = "금액 받는 중"
            print(f"합계 금액은 {total_amount}원입니다.")
    elif state == "금액 받는 중":
        if event == "동전 투입":
            coin = int(input("동전을 투입해주세요: "))
            total_amount += coin
            print(f"합계 금액은 {total_amount}원입니다.")
        elif event == "상품 선택" and total_amount >= price and stock > 0:
            state = "상품 제공"
            total_amount -= price
            stock -= 1
            print("상품을 제공합니다.")
        elif event == "상품 선택":
            print("금액이 부족합니다.")
        elif event == "반환 버튼 누르기":
            state = "반환금 처리"
            print(f"반환 금액은 {total_amount}원입니다.")
            total_amount = 0
    elif state == "상품 제공":
        if event == "상품 받기":
            state = "거스름돈 반환"
            print("상품을 받아 주시기 바랍니다.")
    elif state == "거스름돈 반환":
        if event == "거스름돈 받기":
            state = "초기 상태"
            print(f"거스름돈은 {total_amount}원입니다.")
            total_amount = 0
    elif state == "반환금 처리":
        if event == "반환된 금액 받기":
            state = "초기 상태"
            print(f"거스름돈은 {total_amount}원입니다.")
            total_amount = 0
    else:
        print("오류가 발생했습니다.")

# 주 처리 실행
while True:
    event = input("처리를 선택해주세요 (동전 투입/상품 선택/상품 받기/거스름돈 받기/반환된 금액 받기): ")

    transition(event)

In [ ]:
# 상태 클래스 정의
class VendingMachineState:
    def coin_inserted(self, vm, coin):
        pass

    def select_product(self, vm):
        pass

    def dispense_product(self, vm):
        pass

    def return_change(self, vm):
        pass

    def refund(self, vm):
        pass

# 초기 상태 클래스 정의
class InitialState(VendingMachineState):
    def coin_inserted(self, vm, coin):
        vm.total_amount += coin
        vm.state = vm.amount_received_state
        print(f"합계 금액은 {vm.total_amount}원입니다.")

# 금액 받는 중 상태 클래스 정의
class AmountReceivedState(VendingMachineState):
    def coin_inserted(self, vm, coin):
        vm.total_amount += coin
        print(f"합계 금액은 {vm.total_amount}원입니다.")

    def select_product(self, vm):
        if vm.total_amount >= vm.price and vm.stock > 0:
            vm.state = vm.product_dispensed_state
            vm.total_amount -= vm.price
            vm.stock -= 1
            print("상품을 제공합니다.")
        else:
            print("금액이 부족합니다.")

    def refund(self, vm):
        vm.state = vm.refunding_state
        print(f"반환 금액은 {vm.total_amount}원입니다.")
        vm.total_amount = 0

# 상품 제공 상태 클래스 정의
class ProductDispensedState(VendingMachineState):
    def dispense_product(self, vm):
        vm.state = vm.change_returned_state
        print("상품을 받아 주시기 바랍니다.")

# 거스름돈 반환 상태 클래스 정의
class ChangeReturnedState(VendingMachineState):
    def return_change(self, vm):
        vm.state = vm.initial_state
        print(f"거스름돈은 {vm.total_amount}원입니다.")
        vm.total_amount = 0

# 반환금 처리 상태 클래스 정의
class RefundingState(VendingMachineState):
    def refund(self, vm):
        vm.state = vm.initial_state
        print(f"거스름돈은 {vm.total_amount}원입니다.")
        vm.total_amount = 0

# 자동판매기 클래스 정의
class VendingMachine:
    def __init__(self):
        self.initial_state = InitialState()
        self.amount_received_state = AmountReceivedState()
        self.product_dispensed_state = ProductDispensedState()
        self.change_returned_state = ChangeReturnedState()
        self.refunding_state = RefundingState()
        self.state = self.initial_state
        self.total_amount = 0
        self.price = 1200
        self.stock = 5

    def coin_inserted(self, coin):
        self.state.coin_inserted(self, coin)

    def select_product(self):
        self.state.select_product(self)

    def dispense_product(self):
        self.state.dispense_product(self)

    def return_change(self):
        self.state.return_change(self)

    def refund(self):
        self.state.refund(self)

In [ ]:
# 자동판매기 생성
vm = VendingMachine()

# 초기 상태에서의 처리
vm.coin_inserted(1000) # => 합계 금액은 ___원입니다.

# 금액 받는 중 상태에서의 처리
vm.coin_inserted(500) # => 합계 금액은 ___원입니다.
vm.select_product() # => 금액이 부족합니다.
vm.coin_inserted(1000) # => 합계 금액은 ___원입니다.
vm.select_product() # => 상품을 제공합니다.
vm.dispense_product() # => 상품을 받아 주시기 바랍니다.

# 거스름돈 반환 상태에서의 처리
vm.return_change() # => 거스름돈은 ___원입니다.

# 반환금 처리 상태에서의 처리
vm.coin_inserted(500) # => 합계 금액은 ___원입니다.
vm.refund() # => 거스름돈은 ___원입니다.